# FitchGroup Codeathon '25 - Drive Sustainability Using AI - Group 1

### Problem Understanding

#### What are we predicting?

We're predicting greenhouse gas emissions for companies that don't publicly report them, using two targets:

- **Scope 1 Emissions** - Direct emissions from sources the company owns/controls

    Examples: burning fuel in company vehicles, emissions from company-owned factories, on-site manufacturing processes

- **Scope 2 Emissions** - Indirect emissions from purchased energy

    Examples: electricity bought from the grid, purchased steam/heating/cooling

#### Why does this matter?
Many companies don't report their emissions, making it hard for investors, regulators, and sustainability analysts to assess climate risk. Our models will estimate emissions for these non-reporting companies.

### Key Hypotheses We Can Form

#### Hypothesis 1: Sector Mix Drives Emissions

Different industries have vastly different emission intensities.

Specific predictions:

- Manufacturing, mining, energy → HIGH Scope 1 (they burn fuel, run machinery)
- Utilities, heavy electricity users → HIGH Scope 2 (they buy lots of power)
- Services, software, consulting → LOW both (mostly office work)

Features to test this:

- Percentage in high-emission sectors
- Dominant sector
- Sector diversity

#### Hypothesis 2: Revenue as a Scaling Factor

Bigger companies (by revenue) generally emit more, but emission intensity (emissions per dollar of revenue) varies by sector.

Specific predictions:

- 2x revenue in manufacturing ≈ 2x emissions
- 2x revenue in consulting ≠ 2x emissions (office space doesn't scale linearly)

Features to test this:

- Raw revenue
- Log(revenue) - to handle the wide range
- Interaction: revenue × sector exposure

#### Hypothesis 3: Geography Matters for Scope 2
Scope 2 depends heavily on where you buy electricity because grid carbon intensity varies by country.

Specific predictions:

Same company, same electricity use:

- In coal-heavy grid (e.g., Poland, China) → HIGH Scope 2
- In renewable-heavy grid (e.g., Norway, Iceland) → LOW Scope 2

Features to test this:

- Country/region codes
- Target encoding: average Scope 2 by country

#### Hypothesis 4: Environmental Score Reflects Emissions Management

Companies with better environmental scores likely have emissions reduction programs.

Specific predictions:
- Lower environmental_score (1 = best) → LOWER emissions
- Higher environmental_score (5 = worst) → HIGHER emissions
- This effect is stronger for Scope 1 (more controllable) than Scope 2

Features to test this:
- Environmental score (raw and transformed)
- Environmental activities (positive vs negative adjustments)

#### Hypothesis 5: Scope 1 vs Scope 2 Have Different Drivers
The two targets are fundamentally different and need different features.

Scope 1 (Direct) is driven by:

- Manufacturing intensity
- Fuel combustion activities
- Industrial processes
- Company's own operations

Scope 2 (Indirect) is driven by:

- Electricity consumption
- Grid carbon intensity (geography)
- Office/facility scale
- Purchased energy

Implication: 

We might need:

- Separate models for Scope 1 vs Scope 2, OR
- Multi-output model that learns different weights for each target

#### Hypothesis 6: Revenue Concentration Affects Emission Predictability
Companies focused on one sector are more predictable than diversified conglomerates.

Specific predictions:

- Specialized company (HHI = 10,000): Emissions closely match sector average
- Diversified company (HHI = 2,500): Emissions are averaged across sectors, harder to predict

Features to test this:

- Herfindahl index
- Entropy
- Number of sectors

#### Hypothesis 7: Environmental Activities Are Leading Indicators
Companies actively managing environmental impact will have different emissions than expected from sector alone.

Specific predictions:

- Positive activities (renewable energy, efficiency programs) → LOWER emissions than sector average
- Negative activities (pollution incidents, violations) → HIGHER emissions

Features to test this:

- Count of positive vs negative activities
- Net environmental score adjustment
- Presence of any activities (vs none)

#### Hypothesis 8: SDG Commitments Signal Intent

Companies committed to climate-related SDGs are working on emissions reduction.

Specific predictions:

- Committed to SDG 7 (Clean Energy) or SDG 13 (Climate Action) → LOWER emissions
- More SDG commitments overall → more mature sustainability programs → LOWER emissions

Features to test this:

- Binary flags for climate SDGs
- Total number of SDG commitments

#### Hypothesis 9: Scale sets the baseline

Larger organizations should emit disproportionately more, but only up to a point.
    
Specific predictions:

- log transformed revenue and regional revenue residuals will correlate positively with both scopes, revealing giants and over-performers within each country.

Features to test this:

- `log_revenue`, `log_rev_country_resid`, `logrev_x_highintensity`.

#### Hypothesis 10: Sector mix drives structural emissions

Firms with higher revenue share in carbon-heavy industries should emit above peers even when similar in size.
    
Specific predictions:

- sector concentration, heavy-intensity share, and manufacturing prevalence will show strong positive correlations.

Features to test this:

- `sect_high_intensity_pct`, `sect_C_pct`, `sect_hhi`, `sect_entropy`.

#### Hypothesis 11: Governance gaps signal unmanaged carbon risk

When governance scores outpace environmental scores, execution lags strategy, leading to higher emissions than expected.

Specific predictions:

- governance-environment score gaps correlate with increased scopes, while absolute governance scores correlate negatively.

Features to test this:

- `gov_env_gap`, `governance_score`, `governance_score_squared`.

#### Hypothesis 12: Environmental action mix is a leading indicator

Companies running more positive programs (renewables, efficiency) should under-shoot sector averages, while negative events push emissions higher.

Specific predictions:

- counts/ratios of positive vs. negative activities, net adjustments, and "has activity" flags explain residual variance after sectors.

Features to test this:

- `env_positive_cnt`, `env_positive_ratio`, `env_adj_per_event`, `has_env_activity`.

#### Hypothesis 13: SDG climate commitments amplify or mitigate intensity

Companies that actively pledge SDGs tied to climate and energy (7/12/13) should curb emissions relative to peers with similar industrial exposure.

Specific predictions:

- SDG flags/cluster counts interact with sector intensity to reduce emissions relative to peers.

Features to test this:

- `sdg_climate_focus`, `climate_focus_x_intensity`, `sdg_focus_x_envgap`.

#### Hypothesis 14: Social and governance balance matters

When social and overall ESG momentum outpaces environmental execution, the imbalance should show up as excess emissions unless offset by scale-adjusted investments.

Specific predictions:

- larger positive gaps correlate with higher emissions unless offset by scale-adjusted investments.

Features to test this:

- `env_overall_gap`, `social_env_gap`, `logrev_x_envgap`.

Hypothesis 15: PPP-Adjusted Revenue Reflects True Emission Scale

Raw revenue alone may not accurately reflect operational scale across countries with different cost levels. Adjusting revenue by Purchasing Power Parities (PPP) normalizes for local price differences, giving a better estimate of real production and energy use.

Specific predictions:

Companies in countries with high nominal revenue but low PPP → actual emissions may be lower than raw revenue suggests

Companies in countries with lower nominal revenue but high PPP → actual emissions may be higher than raw revenue suggests

Features to test:

revenue / PPP → PPP-adjusted revenue

Interaction with region (Western Europe, North America) → captures regional energy mix differences

Interaction with sector → aligns scaled revenue with industry-specific emission intensities

Hypothesis 16: Exponentially-Weighted SDG Impact Signals Stronger Emission Reduction Behavior

Companies that commit to high-impact SDGs—and especially those mapped to exponentially larger impact weights—are far more likely to invest in cleaner operations, energy efficiency, and reporting discipline. The exponential scaling captures the idea that not all SDGs contribute equally: a commitment to Climate Action (SDG 13) should matter far more than Peace & Justice (SDG 16).

Specific predictions:

Entities with high exponential SDG impact scores → LOWER Scope 1 and Scope 2 emissions (strong sustainability alignment)

Entities with low or flat impact profiles → HIGHER emissions (minimal environmental commitment)

A single high-impact SDG can outweigh several low-impact ones because exponential weights amplify true environmental focus

Features to test:

Sum of exponential impact indexes per entity (overall Environmental Impact Score)

Average exponential impact index (normalizes by number of SDGs)

Maximum exponential impact index the entity has (captures “strongest” SDG commitment)

Standard deviation of exponential impact indexes (captures whether commitments are concentrated on high-impact environmental goals)

## Import Libraries

In [1]:
# Import Libraries & Requirements
!pip install -r requirements.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

## Load Data Set & Inspection

In [2]:
# Load CSV Data Set
environmental_activities_df = pd.read_csv('data/environmental_activities.csv')
revenue_distribution_by_sector_df = pd.read_csv('data/revenue_distribution_by_sector.csv')
sustainable_development_goals_df = pd.read_csv('data/sustainable_development_goals.csv')
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

### Train Data Set

In [3]:
train_df.head()

,entity_id,region_code,region_name,country_code,country_name,revenue,overall_score,environmental_score,social_score,governance_score,target_scope_1,target_scope_2
0,1782,WEU,Western Europe,GB,United Kingdom of Great Britain and Northern I...,3.528060e+08,2.988,3.900,1.750,2.833,60.0,0.0
1,3918,NAM,Northern America,US,United States of America,1.513700e+09,2.770,3.004,2.942,2.143,265.0,0.0
2,10299,WEU,Western Europe,FR,France,1.560000e+09,2.501,2.979,2.560,1.571,1136.0,0.0
3,2324,NAM,Northern America,US,United States of America,1.238511e+10,3.207,3.776,3.000,2.429,1468.0,0.0
4,1206,WEU,Western Europe,ES,Spain,2.980000e+09,1.998,2.138,1.785,2.000,1802.0,0.0


In [4]:
train_df.shape

(429, 12)

In [5]:
train_df.columns

Index(['entity_id', 'region_code', 'region_name', 'country_code',
       'country_name', 'revenue', 'overall_score', 'environmental_score',
       'social_score', 'governance_score', 'target_scope_1', 'target_scope_2'],
      dtype='object')

In [6]:
train_df.dtypes

entity_id                int64
region_code             object
region_name             object
country_code            object
country_name            object
revenue                float64
overall_score          float64
environmental_score    float64
social_score           float64
governance_score       float64
target_scope_1         float64
target_scope_2         float64
dtype: object

### Test Data Set

In [7]:
test_df.head()

,entity_id,region_code,region_name,country_code,country_name,revenue,overall_score,environmental_score,social_score,governance_score
0,1076,WEU,Western Europe,NL,Netherlands,1.670000e+09,3.170,3.940,2.692,2.357
1,2067,WEU,Western Europe,GB,United Kingdom of Great Britain and Northern I...,5.880000e+08,2.976,4.000,2.014,2.286
2,910,WEU,Western Europe,DE,Germany,1.218100e+09,2.835,3.258,2.955,1.929
3,4082,WEU,Western Europe,DE,Germany,5.037500e+09,2.861,3.360,2.950,1.857
4,4102,WEU,Western Europe,SE,Sweden,1.415400e+09,2.950,3.550,2.900,1.929


In [8]:
test_df.shape

(49, 10)

In [9]:
test_df.columns

Index(['entity_id', 'region_code', 'region_name', 'country_code',
       'country_name', 'revenue', 'overall_score', 'environmental_score',
       'social_score', 'governance_score'],
      dtype='object')

In [10]:
test_df.dtypes

entity_id                int64
region_code             object
region_name             object
country_code            object
country_name            object
revenue                float64
overall_score          float64
environmental_score    float64
social_score           float64
governance_score       float64
dtype: object

### Environmental Activities Data Set

In [11]:
environmental_activities_df.head()

,entity_id,activity_type,activity_code,env_score_adjustment
0,2709,Transportation,M.70.4.P,0.05
1,107,Operation,MTH002,-0.10
2,10045,Operation,MTH002,-0.10
3,2709,End-use,J.58.16.B,-0.05
4,2677,Operation,J.58.20.P,0.10


In [12]:
environmental_activities_df.shape

(355, 4)

In [13]:
environmental_activities_df.columns

Index(['entity_id', 'activity_type', 'activity_code', 'env_score_adjustment'], dtype='object')

In [14]:
environmental_activities_df.dtypes

entity_id                 int64
activity_type            object
activity_code            object
env_score_adjustment    float64
dtype: object

### Revenue Distribution by Sector Data Set

In [15]:
revenue_distribution_by_sector_df.head()

,entity_id,nace_level_1_code,nace_level_1_name,nace_level_2_code,nace_level_2_name,revenue_pct
0,1735,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.031105
1,1195,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.362906
2,4092,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.222190
3,3592,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.063879
4,3169,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.422810


In [16]:
revenue_distribution_by_sector_df.shape

(799, 6)

In [17]:
revenue_distribution_by_sector_df.columns

Index(['entity_id', 'nace_level_1_code', 'nace_level_1_name',
       'nace_level_2_code', 'nace_level_2_name', 'revenue_pct'],
      dtype='object')

In [18]:
revenue_distribution_by_sector_df.dtypes

entity_id              int64
nace_level_1_code     object
nace_level_1_name     object
nace_level_2_code      int64
nace_level_2_name     object
revenue_pct          float64
dtype: object

### Sustainable Development Goals Data Set

In [19]:
sustainable_development_goals_df.head()

,entity_id,sdg_id,sdg_name
0,29,9,"Industry, Innovation and Infrastructure"
1,46,12,Responsible Consumption and Production
2,46,7,Affordable and Clean Energy
3,63,3,Good Health and Wellbeing
4,106,9,"Industry, Innovation and Infrastructure"


In [20]:
sustainable_development_goals_df.shape

(165, 3)

In [21]:
sustainable_development_goals_df.columns

Index(['entity_id', 'sdg_id', 'sdg_name'], dtype='object')

In [22]:
sustainable_development_goals_df.dtypes

entity_id     int64
sdg_id        int64
sdg_name     object
dtype: object

## Data Cleaning

### Check Null Values

In [53]:
# Data Cleaning - Check null values

def analyze_missing_values(df, dataset_name):
    """
    Comprehensive missing value analysis for a dataframe
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe to analyze
    dataset_name : str
        Name of the dataset for display purposes
    
    Returns:
    --------
    pandas DataFrame with missing value statistics
    """
    
    print(f"\n{'='*60}")
    print(f"MISSING VALUE ANALYSIS: {dataset_name}")
    print(f"{'='*60}\n")
    
    # Calculate missing values
    missing_count = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df)) * 100
    dtypes = df.dtypes
    
    # Create summary dataframe
    missing_df = pd.DataFrame({
        'Column': df.columns,
        'Data_Type': dtypes.values,
        'Missing_Count': missing_count.values,
        'Missing_Percentage': missing_pct.values,
        'Non_Missing_Count': len(df) - missing_count.values
    })
    
    # Sort by missing percentage descending
    missing_df = missing_df.sort_values('Missing_Percentage', ascending=False)
    
    # Filter to show only columns with missing values
    missing_cols = missing_df[missing_df['Missing_Count'] > 0]
    
    if len(missing_cols) == 0:
        print("✓ No missing values found in this dataset!\n")
    else:
        print(f"⚠ Found {len(missing_cols)} columns with missing values:\n")
        print(missing_cols.to_string(index=False))
        print(f"\nTotal missing values: {missing_count.sum():,}")
        print(f"Percentage of dataset with any missing value: {(df.isnull().any(axis=1).sum() / len(df) * 100):.2f}%")
    
    return missing_df

In [54]:
train_missing = analyze_missing_values(train_df, "TRAIN_DF")
test_missing = analyze_missing_values(test_df, "TEST_DF")
environmental_activities_missing = analyze_missing_values(environmental_activities_df, "ENVIRONMENTAL_DF")
revenue_distribution_by_sector_missing = analyze_missing_values(revenue_distribution_by_sector_df, "TRAIN_DF")
sustainable_development_goals_missing = analyze_missing_values(sustainable_development_goals_df, "TRAIN_DF")


MISSING VALUE ANALYSIS: TRAIN_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: TEST_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: ENVIRONMENTAL_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: TRAIN_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: TRAIN_DF

✓ No missing values found in this dataset!



### Check Dulplicate Values

In [55]:
# Data Cleaning - Check dulplicate values

def analyze_duplicates(df, dataset_name):
    """
    Comprehensive duplicate row analysis for a dataframe
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe to analyze
    dataset_name : str
        Name of the dataset for display purposes
    
    Returns:
    --------
    pandas DataFrame with duplicate statistics
    """
    
    print(f"\n{'='*60}")
    print(f"DUPLICATE ROW ANALYSIS: {dataset_name}")
    print(f"{'='*60}\n")
    
    # Count full-row duplicates
    duplicate_mask = df.duplicated(keep=False)
    total_duplicates = duplicate_mask.sum()
    unique_duplicate_rows = df[duplicate_mask].drop_duplicates()
    
    if total_duplicates == 0:
        print("✓ No duplicate rows found in this dataset!\n")
        return pd.DataFrame()  # empty result
    
    # Count how many times each duplicated row appears
    duplicate_summary = (
        df[df.duplicated(keep=False)]
        .value_counts()
        .reset_index(name='Count')
        .sort_values('Count', ascending=False)
    )
    
    print(f"⚠ Found {len(unique_duplicate_rows)} unique duplicated rows.")
    print(f"Total duplicated entries (including repeats): {total_duplicates}\n")
    
    print("Most common duplicate patterns:")
    display(duplicate_summary.head(10))
    
    return duplicate_summary


In [56]:
# Run the duplicate-value method on each dataset
train_duplicates = analyze_duplicates(train_df, "TRAIN_DF")
test_duplicates = analyze_duplicates(test_df, "TEST_DF")
environmental_activities_duplicates = analyze_duplicates(environmental_activities_df, "ENVIRONMENTAL_DF")
revenue_distribution_by_sector_duplicates = analyze_duplicates(revenue_distribution_by_sector_df, "REVENUE_DF")
sustainable_development_goals_duplicates = analyze_duplicates(sustainable_development_goals_df, "SDG_DF")


DUPLICATE ROW ANALYSIS: TRAIN_DF

✓ No duplicate rows found in this dataset!


DUPLICATE ROW ANALYSIS: TEST_DF

✓ No duplicate rows found in this dataset!


DUPLICATE ROW ANALYSIS: ENVIRONMENTAL_DF

✓ No duplicate rows found in this dataset!


DUPLICATE ROW ANALYSIS: REVENUE_DF

✓ No duplicate rows found in this dataset!


DUPLICATE ROW ANALYSIS: SDG_DF

✓ No duplicate rows found in this dataset!



### Check Revenue Distribution Coverage

In [57]:
# CHECKING REVENUE DISTRIBUTION COVERAGE

# Get unique entity_ids from each dataset
train_entities = set(train_df['entity_id'].unique())
test_entities = set(test_df['entity_id'].unique())
revenue_entities = set(revenue_distribution_by_sector_df['entity_id'].unique())

print(f"Unique entities in train: {len(train_entities):,}")
print(f"Unique entities in test: {len(test_entities):,}")
print(f"Unique entities in revenue_distribution: {len(revenue_entities):,}")

# Check coverage
train_covered = train_entities & revenue_entities
test_covered = test_entities & revenue_entities

train_missing = train_entities - revenue_entities
test_missing = test_entities - revenue_entities

print("\n" + "="*60)
print("COVERAGE ANALYSIS")
print("="*60)

print(f"\nTRAIN dataset:")
print(f"  Entities WITH revenue data: {len(train_covered):,} ({len(train_covered)/len(train_entities)*100:.2f}%)")
print(f"  Entities WITHOUT revenue data: {len(train_missing):,} ({len(train_missing)/len(train_entities)*100:.2f}%)")

print(f"\nTEST dataset:")
print(f"  Entities WITH revenue data: {len(test_covered):,} ({len(test_covered)/len(test_entities)*100:.2f}%)")
print(f"  Entities WITHOUT revenue data: {len(test_missing):,} ({len(test_missing)/len(test_entities)*100:.2f}%)")

# Check if there are entities in revenue table that aren't in train/test
orphan_entities = revenue_entities - train_entities - test_entities
print(f"\nEntities in revenue table but NOT in train/test: {len(orphan_entities):,}")

print("\n" + "="*60)
print("IMPLICATIONS")
print("="*60)

if len(train_missing) > 0 or len(test_missing) > 0:
    print("\n⚠ IMPORTANT: Some entities don't have revenue distribution data!")
else:
    print("\n✓ Perfect coverage! All entities have revenue distribution data.")


Unique entities in train: 429
Unique entities in test: 49
Unique entities in revenue_distribution: 478

COVERAGE ANALYSIS

TRAIN dataset:
  Entities WITH revenue data: 429 (100.00%)
  Entities WITHOUT revenue data: 0 (0.00%)

TEST dataset:
  Entities WITH revenue data: 49 (100.00%)
  Entities WITHOUT revenue data: 0 (0.00%)

Entities in revenue table but NOT in train/test: 0

IMPLICATIONS

✓ Perfect coverage! All entities have revenue distribution data.


### Checking Sustainable Distribution Coverage

In [58]:
# CHECKING SUSTAINABLE DISTRIBUTION COVERAGE

# Get unique entity_ids from each dataset
sustain_entities = set(sustainable_development_goals_df['entity_id'].unique())

print(f"Unique entities in train: {len(train_entities):,}")
print(f"Unique entities in test: {len(test_entities):,}")
print(f"Unique entities in sustain_distribution: {len(sustain_entities):,}")

# Check coverage
train_covered = train_entities & sustain_entities
test_covered = test_entities & sustain_entities

train_missing = train_entities - sustain_entities
test_missing = test_entities - sustain_entities

print("\n" + "="*60)
print("COVERAGE ANALYSIS")
print("="*60)

print(f"\nTRAIN dataset:")
print(f"  Entities WITH sustainable_development_goals data: {len(train_covered):,} ({len(train_covered)/len(train_entities)*100:.2f}%)")
print(f"  Entities WITHOUT sustainable_development_goals data: {len(train_missing):,} ({len(train_missing)/len(train_entities)*100:.2f}%)")

print(f"\nTEST dataset:")
print(f"  Entities WITH sustainable_development_goals data: {len(test_covered):,} ({len(test_covered)/len(test_entities)*100:.2f}%)")
print(f"  Entities WITHOUT sustainable_development_goals data: {len(test_missing):,} ({len(test_missing)/len(test_entities)*100:.2f}%)")

# Check if there are entities in sustain table that aren't in train/test
orphan_entities = sustain_entities - train_entities - test_entities
print(f"\nEntities in sustain table but NOT in train/test: {len(orphan_entities):,}")

print("\n" + "="*60)
print("IMPLICATIONS")
print("="*60)

if len(train_missing) > 0 or len(test_missing) > 0:
    print("\n⚠ IMPORTANT: Some entities don't have sustain distribution data!")
else:
    print("\n✓ Perfect coverage! All entities have sustain distribution data.")


Unique entities in train: 429
Unique entities in test: 49
Unique entities in sustain_distribution: 130

COVERAGE ANALYSIS

TRAIN dataset:
  Entities WITH sustainable_development_goals data: 118 (27.51%)
  Entities WITHOUT sustainable_development_goals data: 311 (72.49%)

TEST dataset:
  Entities WITH sustainable_development_goals data: 12 (24.49%)
  Entities WITHOUT sustainable_development_goals data: 37 (75.51%)

Entities in sustain table but NOT in train/test: 0

IMPLICATIONS

⚠ IMPORTANT: Some entities don't have sustain distribution data!


In [59]:
print("\n" + "="*60)
print("ISOLATED DATASET OF ENTITIES IN TRAIN DATASET with NO sustainable_development_goals data ")
print("="*60)
Train_To_sustain_distribution_unmatched = (
    train_df
    .merge(sustainable_development_goals_df, on="entity_id", how="left", indicator=True)
    .query("_merge == 'left_only'")
    .drop(columns=["_merge"])
)

print(Train_To_sustain_distribution_unmatched.shape)
Train_To_sustain_distribution_unmatched.head()


ISOLATED DATASET OF ENTITIES IN TRAIN DATASET with NO sustainable_development_goals data 
(311, 14)


,entity_id,region_code,region_name,country_code,country_name,revenue,overall_score,environmental_score,social_score,governance_score,target_scope_1,target_scope_2,sdg_id,sdg_name
1,3918,NAM,Northern America,US,United States of America,1.513700e+09,2.770,3.004,2.942,2.143,265.0,0.0,NaN,NaN
2,10299,WEU,Western Europe,FR,France,1.560000e+09,2.501,2.979,2.560,1.571,1136.0,0.0,NaN,NaN
3,2324,NAM,Northern America,US,United States of America,1.238511e+10,3.207,3.776,3.000,2.429,1468.0,0.0,NaN,NaN
7,1418,NAM,Northern America,US,United States of America,3.588600e+09,2.770,3.083,3.000,1.929,2659.0,0.0,NaN,NaN
9,1494,NAM,Northern America,US,United States of America,1.400000e+10,3.383,3.022,4.900,2.214,4319.0,0.0,NaN,NaN


### Checking Environment Distribution Coverage

In [60]:
# CHECKING ENVIRONMENT DISTRIBUTION COVERAGE

# Get unique entity_ids from each dataset
environment_entities = set(environmental_activities_df['entity_id'].unique())

print(f"Unique entities in train: {len(train_entities):,}")
print(f"Unique entities in test: {len(test_entities):,}")
print(f"Unique entities in environment_distribution: {len(environment_entities):,}")

# Check coverage
train_covered = train_entities & environment_entities
test_covered = test_entities & environment_entities

train_missing = train_entities - environment_entities
test_missing = test_entities - environment_entities

print("\n" + "="*60)
print("COVERAGE ANALYSIS")
print("="*60)
print(f"\nTRAIN dataset:")
print(f"  Entities WITH environment data: {len(train_covered):,} ({len(train_covered)/len(train_entities)*100:.2f}%)")
print(f"  Entities WITHOUT environment data: {len(train_missing):,} ({len(train_missing)/len(train_entities)*100:.2f}%)")

print(f"\nTEST dataset:")
print(f"  Entities WITH environment data: {len(test_covered):,} ({len(test_covered)/len(test_entities)*100:.2f}%)")
print(f"  Entities WITHOUT environment data: {len(test_missing):,} ({len(test_missing)/len(test_entities)*100:.2f}%)")

# Check if there are entities in environment table that aren't in train/test
orphan_entities = environment_entities - train_entities - test_entities
print(f"\nEntities in environment table but NOT in train/test: {len(orphan_entities):,}")

print("\n" + "="*60)
print("IMPLICATIONS")
print("="*60)

if len(train_missing) > 0 or len(test_missing) > 0:
    print("\n⚠ IMPORTANT: Some entities don't have environment distribution data!")
    print("\nWhen merging, you have TWO options:\n")
    
    print("OPTION 1: Left join + Fill missing values with defaults")
    print("  - Use how='left' when merging")
    print("  - Fill NaN values with appropriate defaults:")
    print("    • Numeric features (HHI, percentages): 0 or median")
    print("    • Categorical (dominant sector): 'UNKNOWN' or most common")
    print("  - Create a 'missing_environment_data' flag feature")
    
    print("\nOPTION 2: Inner join (only use entities with complete data)")
    print("  - Use how='inner' when merging")
    print("  - Loses entities without environment data")
    print(f"  - Would lose {len(train_missing)} train and {len(test_missing)} test samples")
    
    print("\n💡 RECOMMENDATION:")
    print("  Use OPTION 1 (left join + imputation) because:")
    print("  - You need to predict ALL test entities")
    print("  - Missing environment data itself might be a signal")
    print("  - Can't afford to lose test samples")
else:
    print("\n✓ Perfect coverage! All entities have environment distribution data.")

Unique entities in train: 429
Unique entities in test: 49
Unique entities in environment_distribution: 260

COVERAGE ANALYSIS

TRAIN dataset:
  Entities WITH environment data: 237 (55.24%)
  Entities WITHOUT environment data: 192 (44.76%)

TEST dataset:
  Entities WITH environment data: 23 (46.94%)
  Entities WITHOUT environment data: 26 (53.06%)

Entities in environment table but NOT in train/test: 0

IMPLICATIONS

⚠ IMPORTANT: Some entities don't have environment distribution data!

When merging, you have TWO options:

OPTION 1: Left join + Fill missing values with defaults
  - Use how='left' when merging
  - Fill NaN values with appropriate defaults:
    • Numeric features (HHI, percentages): 0 or median
    • Categorical (dominant sector): 'UNKNOWN' or most common
  - Create a 'missing_environment_data' flag feature

OPTION 2: Inner join (only use entities with complete data)
  - Use how='inner' when merging
  - Loses entities without environment data
  - Would lose 192 train and 2